In [2]:
from IPython.core.display import HTML
HTML("<style>.container { width:95% !important; }</style>")

In [1]:
%load_ext autoreload
%autoreload 2

# Application of Conformal Prediction Interval Estimations to Market Makers’ Net Positions"
Contains implementation of the paper (WORK IN PROGRESS i.e code is messy= check for errors and  check standarisation  classic or mae).

In short conformal prediction (CP) is compared to quantile regression (QR) and mouving average in an interval prediction task for net position time series. The CP an QR are used on top of 5 popular ML algos:
- Linear Regression with lasso regularisation
- KNN
- Random Forest
- Gradient Boosting
- LSTM

### Plan
#### 1)Load and transform dataset and MA
#### 2)Conformal Prediction
    - Performing Cross Validation and randomized tuning of parameters for considered ML algos
    - Prediction all pvalues all ML algos
#### 3)Quantile Regression
    - Definining Qantile Regression versions for considered ML algos
    - Prediction all pvalues all ML algo 


## 1)Load and transform dataset and MA

In [46]:
import files.prepare as prep

In [42]:
# load dataframe
path_data='data\hourNetPositions.csv'
df = pd.read_csv(path_data,parse_dates=['datetime'], infer_datetime_format=True)
df.columns=['QdfTime', 'NetPosUsd', 'NetUsdPnL', 'AbsVolume', 'Mid']
df=df[['QdfTime','NetPosUsd']]

In [43]:
# see dataframe 
df.head()

,QdfTime,NetPosUsd
0,2014-02-05 01:00:00,-4055.325
1,2014-02-05 02:00:00,-4057.155
2,2014-02-05 03:00:00,-4055.115
3,2014-02-05 04:00:00,-4053.345
4,2014-02-05 05:00:00,-4053.825


In [44]:
# add extra features and split df 
df=prep.addlags(df)
df=prep.market_stats(df)
prep.five_day_split(df)

C:\Users\actio\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:178: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
C:\Users\actio\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:178: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  f"evaluating in Python space because the {repr(op_str)} "
100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:19<00:00,  1.49it/s]


### MA= Mouving average

In [ ]:
import files.mvgavg as ma
import numpy as np

In [ ]:

liste=np.arange(50,2050,50)
alpha=0.05
ma.get__window_ma(df,liste,alpha)

In [ ]:
#we chose the window
window=100
ma.get__window_alpha(df,window)

In [ ]:
### TO DO  plot MA statistics

## 2)Conformal Prediction

##### a) Cross validation and interval prediction for all p values for all algorithms besides LSTM and mouving average

In [ ]:
import files.functionsCV as cv
from tqdm.notebook import tqdm

In [49]:
# CV is long so lets choose only one dataframe
i=0
path= 'data/EURUSD_NETPOSUSD_hourly_regression_split'+str(i)+'.csv'
df=pd.read_csv(path).drop('Unnamed: 0',axis=1) 
df=df.fillna(0)
window=3480

##### KNN

In [ ]:

#knn 
knn_params = {'alpha_':[0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95],
                'n_neighbors':[5,10,25,50,100,150,200],
          'leaf_size':[1,5,10,15,20,30,50],
          'weights':['uniform', 'distance'],
           'randomized_calibration' :[True],
          'n_jobs':[-1],
                'algorithm':['K-NearestNeighbours'] }
cv.grid_cv(df,knn_params)

In [ ]:

all_pvalues_algo('K-NearestNeighbours','CP',window )
all_pvalues_algo('K-NearestNeighbours','NCP',window )

##### Linear regression lasso

In [ ]:
lasso_params = {'alpha_':[0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95],
                'alpha':[25,10,5,3,2,1.0,0.0,0.1,0.01,0.001],
                'randomized_calibration' :[True],
                'algorithm':['LassoRegression'] }
cv.grid_cv(df,lasso_params)

In [ ]:
all_pvalues_algo('LassoRegression','CP',window )
all_pvalues_algo('LassoRegression','NCP',window )

#####  Gradient boosting

In [ ]:
boosting_params = {'alpha_':[0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95],
               
     'n_estimators': [50,100, 200 ],
     "criterion": [  "mse"],
    "max_features": [ 0.1,   0.3,   0.5,  0.7,   0.9, 1.0],
    "min_samples_split": [2, 5, 10,25],
    "min_samples_leaf": [1,5,10,20],
    'randomized_calibration' :[True],
    "bagging_freq": [0, 1, 3],
          'n_jobs':[-1],
                'algorithm':['GradientBoosting' ]}
cv.grid_cv(df,boosting_params)

In [ ]:
all_pvalues_algo('GradientBoosting','CP',window )
all_pvalues_algo('GradientBoosting','NCP',window )

##### random forest

In [ ]:
rf_params = {'alpha_':[0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95],
                'n_estimators': [50,100, 200 ],
                 "criterion": [  "mse"],
                "max_features": [ 0.1,   0.3,   0.5,  0.7,   0.9, 1.0],
                "min_samples_split": [2, 5, 10,25],
                "min_samples_leaf": [1,5,10,20],
                'randomized_calibration' :[True],
                'n_jobs':[-1],
                'algorithm':['RandomForest'] }
cv.grid_cv(df,fr_params)

In [ ]:
all_pvalues_algo('RandomForest','CP',window )
all_pvalues_algo('RandomForest','NCP',window )

##### b) Cross validation and interval prediction for all p values for   LSTM TODO

#### c) LSTM

In [ ]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
set_session(sess)

In [ ]:
import files.cudalstm as lstm

In [ ]:
lstm.trainlstm()

# 3)Quantile Regression

In [ ]:
tune=True

##### random forest

In [ ]:
all_pvalues_algo('QuantileRandomForest',3480,tune)

##### Linear regression lasso

In [ ]:
all_pvalues_algo('QuantileRegression',3480,tune)

##### KNN

In [ ]:
all_pvalues_algo('QuantileKNN',3480,tune)

#####  Gradient boosting

In [ ]:
all_pvalues_algo('QuantileGradientBoosting',3480,tune)

##### LSTM

In [ ]:
import files.DeepQuantileRegressionCINetUsdPospaper as qlstm

In [ ]:
qlstm.trainqlstm()